In [2]:
from datetime import date

import pandas as pd

from backtest_engine import BacktestConfig, BacktestEngine
from backtest_engine.config import (
    StudyWindow,
    UniverseConfig,
    SignalConfig,
    SelectionConfig,
    RebalanceConfig,
)


# 1. Dummy DataProvider for now (so we can see that the engine runs)
from backtest_engine.data_provider import DataProvider


class DummyProvider(DataProvider):
    def get_universe(self, as_of, universe_config):
        return pd.DataFrame(
            {
                "schemecode": [1, 2, 3, 4, 5],
                "scheme_name": [f"Fund {i}" for i in range(1, 6)],
            }
        )

    def get_signal_scores(self, as_of, schemecodes, signal_config):
        return pd.DataFrame(
            {"schemecode": schemecodes, "score": [5, 4, 3, 2, 1]}
        )

    def get_nav_series(self, schemecodes, start, end):
        raise NotImplementedError

    def get_benchmark_series(self, start, end):
        raise NotImplementedError

config = BacktestConfig(
    name="dummy-test",
    study_window=StudyWindow(start=date(2020, 1, 1), end=date(2021, 1, 1)),
    universe=UniverseConfig(preset="equity_active_direct"),
    signal=SignalConfig(name="rank_12m_category"),
    selection=SelectionConfig(
        top_n=3,
        min_funds=3,   # 👈 lowered for dummy test so engine doesn't raise
    ),
    rebalance=RebalanceConfig(frequency="12M"),
)

engine = BacktestEngine(DummyProvider())
result = engine.run(config)

result.summary, result.portfolio_periods, result.holdings.head()


(       run_id        name rebalance_date  num_funds
 0  dummy-test  dummy-test     2020-01-01          3,
        run_id  period_no rebalance_date  start_date    end_date  num_funds
 0  dummy-test          1     2020-01-01  2020-01-01  2021-01-01          3,
    schemecode scheme_name  score    weight      run_id  period_no  \
 0           1      Fund 1      5  0.333333  dummy-test          1   
 1           2      Fund 2      4  0.333333  dummy-test          1   
 2           3      Fund 3      3  0.333333  dummy-test          1   
 
   rebalance_date  
 0     2020-01-01  
 1     2020-01-01  
 2     2020-01-01  )